In [ ]:
import requests
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
import time
from bs4 import BeautifulSoup
import requests
import bs4
import os
import aswan

In [ ]:
link = "https://merce.hu/wp-sitemap-posts-post-9.xml"
resp = requests.get(link)
soup = BeautifulSoup(resp.content)

In [ ]:
links = [item.get_text() for item in soup.find_all("loc")]
dates = [ i[i.find("2021"):i.find("2021")+10] for i in links ]

In [ ]:
url_merce = (
    pd.DataFrame(links, dates)
    .reset_index()
    .rename({"index" : "date", 0: "url"}, axis = 1)
    .sort_values(by = ["date"])
)
url_merce["date"] = pd.to_datetime(url_merce["date"])
url_merce.head()

In [ ]:
url_merce.to_pickle("links/merce_cikk_linkek.pkl")

### Parser

In [ ]:
link = "https://merce.hu/2021/08/15/a-talibok-benyomultak-kabul-elovarosaba-boris-johnson-rendkivuli-parlamenti-ulest-hivott-ossze/"
resp = requests.get(link)
soup = BeautifulSoup(resp.content)
print(link)

In [ ]:
def parse_soup(self, soup):

    try:
        title = soup.find(attrs={"class": "entry-title"}).get_text().replace("\n", "")
    except:
        title = ""

    try:
        text = " ".join(
            [
                item.get_text().strip()
                for item in soup.find(attrs={"class": "entry-content"}).find_all("p")
            ]
        )
    except:
        text = ""
    return {
        "title": title,
        "text": text,
    }

In [ ]:
parse_soup("a",soup)

In [ ]:
import ray

In [ ]:
ray.shutdown()

config = aswan.AswanConfig.default_from_dir("data_merce")

text_table = config.get_prod_table("text")

project = aswan.Project(config) 

@project.register_handler
class TextHandler(aswan.UrlHandler):
    def parse_soup(self, soup):

        try:
            title = soup.find(attrs={"class": "entry-title"}).get_text().replace("\n", "")
        except:
            title = ""
    
        try:
            text = " ".join(
                [
                    item.get_text().strip()
                    for item in soup.find(attrs={"class": "entry-content"}).find_all("p")
                ]
            )
        except:
            text = ""
        return {
            "title": title,
            "text": text,
        }

project.register_t2_table(text_table)

@project.register_t2_integrator
class TextIntegrator(aswan.FlexibleDfParser):
    handlers = [TextHandler]

    def url_parser(self, url):
        return {"id": url.split("/")[-1]}

    def get_t2_table(self):
        return url_table

def add_init_urls():
    arcive_urls = url_merce.url.tolist().copy()

    project.add_urls_to_handler(TextHandler, arcive_urls)

In [ ]:
add_init_urls()

In [ ]:
project.config.prod.batch_size = 20
project.config.prod.min_queue_size = 200

In [ ]:
project.run()

In [ ]:
y
